In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation, FFMpegWriter

# Допустим, eye_sample — это Raw-объект с каналами 'x_pos' и 'y_pos'
eye_sample = eye_raw.copy().crop(tmax=250, include_tmax=False)

# Извлечем данные
data, times = eye_sample.copy().pick(['EOG_x', 'EOG_y']).get_data(return_times=True)

x = np.nan_to_num(data[0], nan=-100)
y = np.nan_to_num(data[1], nan=-100)

# Настройки
x_res, y_res = 1920, 1080  # или 960x540 если нужно меньше
step = 30  # рисуем каждый 30-й кадр
fps = 30   # сохраняем анимацию с 30 кадров в секунду
max_frames = len(x) // step  # можно ограничить, если надо

# Прореживаем и масштабируем
x_scaled = (x - np.min(x)) / (np.max(x) - np.min(x)) * x_res
y_scaled = (y - np.min(y)) / (np.max(y) - np.min(y)) * y_res
x_sub = x_scaled[::step]
y_sub = y_scaled[::step]
times_sub = times[::step]

# Интервал между кадрами в мс для отображения (но сохраняем как 30 fps)
interval_ms = 1000 / fps

# Настраиваем график
fig, ax = plt.subplots()
point, = ax.plot([], [], 'ro')
ax.set_xlim(0, x_res)
ax.set_ylim(0, y_res)
ax.invert_yaxis()
title = ax.set_title("Eyetracking: 0.00s")
ax.set_xlabel("X")
ax.set_ylabel("Y")


# Подготовим список блоков с интервалами времени и типом
blocks = []
block_items = list(experiment_seq.values())
for i, block in enumerate(block_items):
    start = block['timestamp']
    end = block_items[i + 1]['timestamp'] if i + 1 < len(block_items) else times[-1]
    blocks.append({'start': start, 'end': end, 'type': block['type']})
    

def update(i):
    current_time = times_sub[i] - times_sub[0]

    # Определяем текущий блок по времени
    block_type = "unknown"
    for block in blocks:
        if block['start'] <= current_time < block['end']:
            block_type = block['type']
            break

    point.set_data([x_sub[i]], [y_sub[i]])
    title.set_text(f"Eyetracking: {current_time:.2f}s | Block type: {block_type}")
    return point, title

ani = FuncAnimation(fig, update, frames=len(x_sub), interval=interval_ms, blit=True)

# Сохраняем в MP4
writer = FFMpegWriter(fps=fps)
ani.save("eog_animation_30fps.mp4", writer=writer)

print("Анимация сохранена: eog_animation_30fps.mp4")